In [216]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import datetime
from dateutil.relativedelta import relativedelta

In [217]:
end_date_for_ind_variable = datetime.datetime(2018, 8, 4)
campaign_launched = datetime.datetime(2018, 9, 4)
end_date_tar_variable = datetime.datetime(2019, 6, 18)
campaign_id = 6169

# Reading Data

In [218]:
campaigns = pd.read_csv('./data/campaigns.csv')

donors = pd.read_csv('./data/donors.csv')

gifts = pd.read_csv('./data/gifts.csv')

campaigns_campaign_id = pd.read_csv('./data/selection campaign 6169.csv')

# GIFTS

### Checking data in gifts

In [219]:

print(gifts.isna().sum().sum())
for column in gifts.columns[0:]:
    print(column + " " + str(gifts[column].isna().sum()))

gifts.info()

15637
Unnamed: 0 0
donorID 0
campaignID 15637
amount 0
date 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216594 entries, 0 to 216593
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  216594 non-null  int64  
 1   donorID     216594 non-null  int64  
 2   campaignID  200957 non-null  float64
 3   amount      216594 non-null  object 
 4   date        216594 non-null  object 
dtypes: float64(1), int64(2), object(2)
memory usage: 8.3+ MB


### Cleaning data in gifts

In [220]:
gifts['campaignID'] = gifts['campaignID'].fillna(0.0)
gifts['campaignID'] = gifts['campaignID'].astype(int)
gifts['amount'] = gifts['amount'].str.replace(",", ".").astype(float)
gifts['date'] = pd.to_datetime(gifts['date'])
gifts.rename(columns={'Unnamed: 0' : 'giftID'}, inplace=True)
gifts.head()

,giftID,donorID,campaignID,amount,date
0,0,100001,0,12.39,2004-03-23
1,1,100001,1577,12.39,2007-10-11
2,2,100001,1634,6.69,2007-12-28
3,3,100001,1778,24.79,2008-06-14
4,4,100001,1808,7.44,2008-07-09


In [221]:
gifts['Quater'] = np.where(gifts['date'].dt.month < 4, 'Q1', np.where(gifts['date'].dt.month < 7, 'Q2', np.where(gifts['date'].dt.month < 10, 'Q3','Q4')))
gifts.head()

,giftID,donorID,campaignID,amount,date,Quater
0,0,100001,0,12.39,2004-03-23,Q1
1,1,100001,1577,12.39,2007-10-11,Q4
2,2,100001,1634,6.69,2007-12-28,Q4
3,3,100001,1778,24.79,2008-06-14,Q2
4,4,100001,1808,7.44,2008-07-09,Q3


# DONORS

### Checking data in donors

In [222]:
print(donors.isna().sum().sum())
for column in donors.columns[0:]:
    print(column + " " + str(donors[column].isna().sum()))
donors.head()

34
Unnamed: 0 0
donorID 0
zipcode 2
province 0
region 0
gender 32
language 0
dateOfBirth 0


,Unnamed: 0,donorID,zipcode,province,region,gender,language,dateOfBirth
0,0,100001,1861.0,Flemish Brabant,Flanders,F,NL,12/24/1967
1,1,100002,2260.0,Antwerp,Flanders,M,NL,01/14/1952
2,2,100003,1780.0,Flemish Brabant,Flanders,M,NL,12/23/1986
3,3,100004,1020.0,Brussels,Brussels,F,FR,03/29/1952
4,4,100005,1020.0,Brussels,Brussels,F,FR,06/25/1986


### Cleaning data in donors

In [223]:
donors_zipcode_na = donors.loc[donors['zipcode'].isna(), :]
donors_zipcode_na.head()
donors['zipcode'] = donors['zipcode'].fillna(0)

In [224]:
donors_gender_na = donors.loc[donors['gender'].isna(), :]
donors_gender_na.head()
donors['gender'] = donors['gender'].fillna('Not Known')

In [225]:
donors.drop(['Unnamed: 0'] , axis = 1, inplace=True)
donors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44691 entries, 0 to 44690
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   donorID      44691 non-null  int64  
 1   zipcode      44691 non-null  float64
 2   province     44691 non-null  object 
 3   region       44691 non-null  object 
 4   gender       44691 non-null  object 
 5   language     44691 non-null  object 
 6   dateOfBirth  44691 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 2.4+ MB


# CAMPAIGNS

### Checking data in campaigns

In [226]:
print(campaigns.isna().sum().sum())
for column in campaigns.columns[0:]:
    print(column + " " + str(campaigns[column].isna().sum()))

campaigns.info()

0
campaignID;date;lettersSent;CostUnit 0
<class 'pandas.core.frame.DataFrame'>
Index: 367 entries, 153;13/12/2004;6 873;0 to 7536;15/12/2019;37 187;0
Data columns (total 1 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   campaignID;date;lettersSent;CostUnit  367 non-null    object
dtypes: object(1)
memory usage: 5.7+ KB


In [227]:
campaigns.head()

,campaignID;date;lettersSent;CostUnit
153;13/12/2004;6 873;0,17 €
154;01/01/2005;7 656;0,30 €
180;18/01/2005;9 933;0,20 €
433;29/01/2005;7 448;0,59 €
476;12/02/2005;6 605;0,51 €


### Cleaning data in campaigns

In [228]:
campaigns = pd.read_csv('./data/campaigns.csv')
campaigns.rename(columns={campaigns.columns[0]: "Cost" }, inplace = True)
campaigns.index.names = ['campaignID;date;lettersSent;CostUnit']
campaigns_reset = campaigns.reset_index()
campaigns_split = campaigns_reset['campaignID;date;lettersSent;CostUnit'].str.split(';', expand=True)
campaigns_split['cost'] = campaigns_reset['Cost']
campaigns_split.rename(columns={0: "campaignID" }, inplace = True)
campaigns_split.rename(columns={1: "date" }, inplace = True)
campaigns_split.rename(columns={2: "lettersSent" }, inplace = True)
campaigns_split.drop([3] , axis = 1, inplace=True)
campaigns_split.head()

,campaignID,date,lettersSent,cost
0,153,13/12/2004,6 873,17 €
1,154,01/01/2005,7 656,30 €
2,180,18/01/2005,9 933,20 €
3,433,29/01/2005,7 448,59 €
4,476,12/02/2005,6 605,51 €


In [229]:
campaigns_split['campaignID'] = campaigns_split['campaignID'].astype(int)
campaigns_split['date'] = pd.to_datetime(campaigns_split['date'])
campaigns_split['lettersSent'] = campaigns_split['lettersSent'].str.replace("\u202f", "").astype(int)
campaigns_split['cost'] = campaigns_split['cost'].str.replace("\xa0€", "").astype(int)
campaigns = campaigns_split
campaigns.head()


,campaignID,date,lettersSent,cost
0,153,2004-12-13,6873,17
1,154,2005-01-01,7656,30
2,180,2005-01-18,9933,20
3,433,2005-01-29,7448,59
4,476,2005-12-02,6605,51


In [230]:
campaigns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   campaignID   367 non-null    int32         
 1   date         367 non-null    datetime64[ns]
 2   lettersSent  367 non-null    int32         
 3   cost         367 non-null    int32         
dtypes: datetime64[ns](1), int32(3)
memory usage: 7.3 KB


### Creating Features

In [231]:
#Calculating summary for donors till Aug 4th 2018
gifts_donor_campaignID = gifts[gifts['date'] < end_date_for_ind_variable].groupby('donorID', as_index=False)['campaignID'].agg({'number_of_campaigns':'count'})
gifts_donor_total_amount = gifts[gifts['date'] < end_date_for_ind_variable].groupby('donorID', as_index=False)['amount'].agg({'total_amount':'sum'})
gifts_donor_last_date = gifts[gifts['date'] < end_date_for_ind_variable].groupby('donorID', as_index=False)['date'].agg({'last_date':'max'})
gifts_donor_last_date = gifts[gifts['date'] < end_date_for_ind_variable].groupby('donorID', as_index=False)['date'].agg({'last_date':'max'})
gifts_donor_quater_contribution = gifts[gifts['date'] < end_date_for_ind_variable].groupby(['donorID', 'Quater'], as_index=False)['date'].agg({'count'}).unstack('Quater')


donors = donors.merge(gifts_donor_campaignID, left_on = 'donorID', right_on = 'donorID', how = 'left')
donors = donors.merge(gifts_donor_total_amount, left_on = 'donorID', right_on = 'donorID', how = 'left')
donors = donors.merge(gifts_donor_last_date, left_on = 'donorID', right_on = 'donorID', how = 'left')
donors = donors.merge(gifts_donor_quater_contribution, left_on = 'donorID', right_on = 'donorID', how = 'left')

donors['days_since_contributed'] = end_date_for_ind_variable - donors['last_date'] 
donors.head() 

C:\tools\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:648: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


,donorID,zipcode,province,region,gender,language,dateOfBirth,number_of_campaigns,total_amount,last_date,"(count, Q1)","(count, Q2)","(count, Q3)","(count, Q4)",days_since_contributed
0,100001,1861.0,Flemish Brabant,Flanders,F,NL,12/24/1967,12.0,178.49,2011-12-30,3.0,3.0,1.0,5.0,2409 days
1,100002,2260.0,Antwerp,Flanders,M,NL,01/14/1952,1.0,20.00,2015-01-10,1.0,NaN,NaN,NaN,1302 days
2,100003,1780.0,Flemish Brabant,Flanders,M,NL,12/23/1986,5.0,18.00,2018-07-08,NaN,1.0,4.0,NaN,27 days
3,100004,1020.0,Brussels,Brussels,F,FR,03/29/1952,26.0,716.22,2011-06-03,8.0,4.0,4.0,10.0,2619 days
4,100005,1020.0,Brussels,Brussels,F,FR,06/25/1986,4.0,14.96,2018-06-19,NaN,1.0,NaN,3.0,46 days


In [232]:
#Calculating summary for donors till Aug 4th 2018
gifts_donor_summary = gifts[gifts['date'] < end_date_for_ind_variable].groupby(['donorID'], as_index=False)['amount'].agg(['mean', 'min', 'max'])
gifts_donor_quater_spending = gifts[gifts['date'] < end_date_for_ind_variable].groupby(['donorID', 'Quater'], as_index=False)['amount'].agg({'mean', 'min', 'max', 'sum'}).unstack('Quater')

donors = donors.merge(gifts_donor_summary, left_on = 'donorID', right_on = 'donorID', how = 'left')
donors = donors.merge(gifts_donor_quater_spending, left_on = 'donorID', right_on = 'donorID', how = 'left')
donors.head()

C:\tools\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:648: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


,donorID,zipcode,province,region,gender,language,dateOfBirth,number_of_campaigns,total_amount,last_date,...,"(min, Q3)","(min, Q4)","(mean, Q1)","(mean, Q2)","(mean, Q3)","(mean, Q4)","(max, Q1)","(max, Q2)","(max, Q3)","(max, Q4)"
0,100001,1861.0,Flemish Brabant,Flanders,F,NL,12/24/1967,12.0,178.49,2011-12-30,...,7.44,6.69,14.130,21.596667,7.44,12.774,20.0,30.0,7.44,24.79
1,100002,2260.0,Antwerp,Flanders,M,NL,01/14/1952,1.0,20.00,2015-01-10,...,NaN,NaN,20.000,NaN,NaN,NaN,20.0,NaN,NaN,NaN
2,100003,1780.0,Flemish Brabant,Flanders,M,NL,12/23/1986,5.0,18.00,2018-07-08,...,3.00,NaN,NaN,4.000000,3.50,NaN,NaN,4.0,5.00,NaN
3,100004,1020.0,Brussels,Brussels,F,FR,03/29/1952,26.0,716.22,2011-06-03,...,24.79,24.79,27.395,28.697500,24.79,28.311,30.0,30.0,24.79,49.58
4,100005,1020.0,Brussels,Brussels,F,FR,06/25/1986,4.0,14.96,2018-06-19,...,NaN,2.48,NaN,5.000000,NaN,3.320,NaN,5.0,NaN,5.00


In [233]:
#Calculating age for donors till Aug 4th 2018
donors.loc[:,'age'] = donors['dateOfBirth'].apply(lambda x : end_date_for_ind_variable.year - pd.to_datetime(x).year)
donors.loc[:,'age-group'] = donors['age'] // 10 * 10
donors.head()

,donorID,zipcode,province,region,gender,language,dateOfBirth,number_of_campaigns,total_amount,last_date,...,"(mean, Q1)","(mean, Q2)","(mean, Q3)","(mean, Q4)","(max, Q1)","(max, Q2)","(max, Q3)","(max, Q4)",age,age-group
0,100001,1861.0,Flemish Brabant,Flanders,F,NL,12/24/1967,12.0,178.49,2011-12-30,...,14.130,21.596667,7.44,12.774,20.0,30.0,7.44,24.79,51,50
1,100002,2260.0,Antwerp,Flanders,M,NL,01/14/1952,1.0,20.00,2015-01-10,...,20.000,NaN,NaN,NaN,20.0,NaN,NaN,NaN,66,60
2,100003,1780.0,Flemish Brabant,Flanders,M,NL,12/23/1986,5.0,18.00,2018-07-08,...,NaN,4.000000,3.50,NaN,NaN,4.0,5.00,NaN,32,30
3,100004,1020.0,Brussels,Brussels,F,FR,03/29/1952,26.0,716.22,2011-06-03,...,27.395,28.697500,24.79,28.311,30.0,30.0,24.79,49.58,66,60
4,100005,1020.0,Brussels,Brussels,F,FR,06/25/1986,4.0,14.96,2018-06-19,...,NaN,5.000000,NaN,3.320,NaN,5.0,NaN,5.00,32,30


In [234]:
gifts.head()

,giftID,donorID,campaignID,amount,date,Quater
0,0,100001,0,12.39,2004-03-23,Q1
1,1,100001,1577,12.39,2007-10-11,Q4
2,2,100001,1634,6.69,2007-12-28,Q4
3,3,100001,1778,24.79,2008-06-14,Q2
4,4,100001,1808,7.44,2008-07-09,Q3


In [235]:
campaigns_campaign_id.nunique()

donorID    34888
dtype: int64

In [236]:
selected_donors = donors.merge(campaigns_campaign_id, left_on='donorID', right_on='donorID', how = 'inner')
selected_donors.head()
selected_donors.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34874 entries, 0 to 34873
Data columns (total 36 columns):
 #   Column                  Non-Null Count  Dtype          
---  ------                  --------------  -----          
 0   donorID                 34874 non-null  int64          
 1   zipcode                 34874 non-null  float64        
 2   province                34874 non-null  object         
 3   region                  34874 non-null  object         
 4   gender                  34874 non-null  object         
 5   language                34874 non-null  object         
 6   dateOfBirth             34874 non-null  object         
 7   number_of_campaigns     34869 non-null  float64        
 8   total_amount            34869 non-null  float64        
 9   last_date               34869 non-null  datetime64[ns] 
 10  (count, Q1)             18769 non-null  float64        
 11  (count, Q2)             18686 non-null  float64        
 12  (count, Q3)             13955 no

In [237]:
donors 

,donorID,zipcode,province,region,gender,language,dateOfBirth,number_of_campaigns,total_amount,last_date,...,"(mean, Q1)","(mean, Q2)","(mean, Q3)","(mean, Q4)","(max, Q1)","(max, Q2)","(max, Q3)","(max, Q4)",age,age-group
0,100001,1861.0,Flemish Brabant,Flanders,F,NL,12/24/1967,12.0,178.49,2011-12-30,...,14.130,21.596667,7.44,12.774,20.0,30.00,7.44,24.79,51,50
1,100002,2260.0,Antwerp,Flanders,M,NL,01/14/1952,1.0,20.00,2015-01-10,...,20.000,NaN,NaN,NaN,20.0,NaN,NaN,NaN,66,60
2,100003,1780.0,Flemish Brabant,Flanders,M,NL,12/23/1986,5.0,18.00,2018-07-08,...,NaN,4.000000,3.50,NaN,NaN,4.00,5.00,NaN,32,30
3,100004,1020.0,Brussels,Brussels,F,FR,03/29/1952,26.0,716.22,2011-06-03,...,27.395,28.697500,24.79,28.311,30.0,30.00,24.79,49.58,66,60
4,100005,1020.0,Brussels,Brussels,F,FR,06/25/1986,4.0,14.96,2018-06-19,...,NaN,5.000000,NaN,3.320,NaN,5.00,NaN,5.00,32,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44686,144687,4900.0,Liege,Wallonia,M,FR,09/12/1970,1.0,10.00,2015-11-06,...,NaN,NaN,NaN,10.000,NaN,NaN,NaN,10.00,48,40
44687,144688,3600.0,Limburg,Flanders,M,NL,11/28/1972,7.0,48.56,2017-07-27,...,5.000,5.590000,15.00,6.200,5.0,7.44,15.00,6.20,46,40
44688,144689,3061.0,Flemish Brabant,Flanders,M,NL,04/21/1967,2.0,124.79,2018-06-25,...,NaN,100.000000,24.79,NaN,NaN,100.00,24.79,NaN,51,50
44689,144690,1950.0,Flemish Brabant,Flanders,M,FR,05/07/1976,1.0,40.00,2014-12-30,...,NaN,NaN,NaN,40.000,NaN,NaN,NaN,40.00,42,40


In [238]:
# Checking for mutiple donations for campaign campaign_id
gifts1 = gifts[gifts['campaignID'] == campaign_id]
agg_gifts = gifts1.groupby(['donorID'])['date'].agg(['count']).sort_values(by = 'count', ascending =False)


In [239]:
selected_donors = selected_donors.merge(gifts1, left_on='donorID', right_on='donorID', how = 'left')
# If Contribution made after May 18 

In [240]:
selected_donors['amount'].fillna(0)
#selected_donors.loc[(selected_donors['amount'] > 30) and (selected_donors['date'] > campaign_launched) and (selected_donors['date'] < end_date_tar_variable), 'has_contributed'] = 1

selected_donors['has_contributed'] = np.where((selected_donors['amount'] > 30) & (selected_donors['date'] > campaign_launched) & (selected_donors['date'] < end_date_tar_variable),1, 0)

In [241]:
for column in selected_donors[0:]:
    print(column)

donorID
zipcode
province
region
gender
language
dateOfBirth
number_of_campaigns
total_amount
last_date
('count', 'Q1')
('count', 'Q2')
('count', 'Q3')
('count', 'Q4')
days_since_contributed
mean
min
max
('sum', 'Q1')
('sum', 'Q2')
('sum', 'Q3')
('sum', 'Q4')
('min', 'Q1')
('min', 'Q2')
('min', 'Q3')
('min', 'Q4')
('mean', 'Q1')
('mean', 'Q2')
('mean', 'Q3')
('mean', 'Q4')
('max', 'Q1')
('max', 'Q2')
('max', 'Q3')
('max', 'Q4')
age
age-group
giftID
campaignID
amount
date
Quater
has_contributed


In [242]:
drop_columns = ['zipcode', 'dateOfBirth', 'last_date', 'giftID', 'campaignID', 'Quater', 'date']
selected_donors = selected_donors.drop(drop_columns, axis = 1)


In [243]:
selected_donors = selected_donors.rename(columns={('count', 'Q1') : 'Q1_Donations', ('count', 'Q2') : 'Q2_Donations', ('count', 'Q3') : 'Q3_Donations', ('count', 'Q4') : 'Q4_Donations'})


In [244]:
selected_donors = selected_donors.rename(columns={('mean', 'Q1') : 'Q1_Mean', ('mean', 'Q2') : 'Q2_Mean', ('mean', 'Q3') : 'Q3_Mean', ('mean', 'Q4') : 'Q4_Mean'})

In [245]:
selected_donors = selected_donors.rename(columns={('min', 'Q1') : 'Q1_Min', ('min', 'Q2') : 'Q2_Min', ('min', 'Q3') : 'Q3_Min', ('min', 'Q4') : 'Q4_Min'})

In [246]:
selected_donors = selected_donors.rename(columns={('max', 'Q1') : 'Q1_Max', ('max', 'Q2') : 'Q2_Max', ('max', 'Q3') : 'Q3_Max', ('max', 'Q4') : 'Q4_Max'})

In [247]:
selected_donors = selected_donors.rename(columns={('sum', 'Q1') : 'Q1_Amount', ('sum', 'Q2') : 'Q2_Amount', ('sum', 'Q3') : 'Q3_Amount', ('sum', 'Q4') : 'Q4_Amount'})

In [248]:
for column in selected_donors[0:]:
    print(column)

donorID
province
region
gender
language
number_of_campaigns
total_amount
Q1_Donations
Q2_Donations
Q3_Donations
Q4_Donations
days_since_contributed
mean
min
max
Q1_Amount
Q2_Amount
Q3_Amount
Q4_Amount
Q1_Min
Q2_Min
Q3_Min
Q4_Min
Q1_Mean
Q2_Mean
Q3_Mean
Q4_Mean
Q1_Max
Q2_Max
Q3_Max
Q4_Max
age
age-group
amount
has_contributed
